In [ ]:
cd /home/yuchen/pulse2percept

In [ ]:
import shapes
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pulse2percept as p2p
import skimage
from skimage.measure import label, regionprops, regionprops_table
import math
import string
from statistics import mean
from sklearn.linear_model import LinearRegression
import shapely.geometry as geom
import statsmodels.api as sm 
import pingouin as pg
from statsmodels.stats.outliers_influence import variance_inflation_factor
from matplotlib.ticker import FormatStrFormatter, MaxNLocator, MultipleLocator

In [ ]:
def curve_length(curve):
    return np.sum(np.sqrt(np.sum((curve[:-1] - curve[1:])**2,axis=1)))

# Analysis

In [ ]:
# calculate shapes 
data = shapes.load_shapes("/home/yuchen/shapes/data/shapes.h5", subjects=['12-005','51-009','52-001'],stim_class=None)

result = pd.DataFrame({})
for i in range(len(data)):
    label_img = skimage.measure.label(data['image'][i]>0)
    regions = regionprops(label_img)
    props = regionprops_table(label_img, properties=('centroid',
                                                     'orientation',
                                                     'major_axis_length',
                                                     'minor_axis_length',
                                                     'area',
                                                    'eccentricity',
                                                    'perimeter'))
    df = pd.DataFrame(props).astype('object')
    df.at[0,'centroid-0'] = df.iloc[:, 0].tolist()  # store centroid-x
    df.at[0,'centroid-1'] = df.iloc[:, 1].tolist()  # store centroid-y
    df.at[0,'orientation'] = df.iloc[:, 2].tolist()  # store orientation
    df.at[0,'major_axis_length'] = df.iloc[:, 3].tolist()  # major
    df.at[0,'minor_axis_length'] = df.iloc[:, 4].tolist()  # minor
    df.at[0,'area'] = df.iloc[:, 5].tolist()  # area
    df.at[0,'eccentricity'] = df.iloc[:, 6].tolist()  
    df.at[0,'perimeter'] = df.iloc[:, 7].tolist()  
    result = pd.concat([result, df.iloc[:1,:]],axis=0)

result = result.rename(columns={"area":"size", "orientation":"orientation_new", "eccentricity":"eccentricity_new" })
data = pd.concat([data,result.reset_index(drop=True)],axis=1)
data_copy = data.copy()

In [ ]:
data

In [ ]:
data = data_copy.copy()

data_double = data[(data['electrode2'] != str())].reset_index(drop=True)
data_single = data[(data['electrode2'] == str())].reset_index(drop=True)
data_single['stim_class'] ='SingleElectrode'
df = data_single[['subject', 'electrode1', 'freq', 'amp1']].drop_duplicates().reset_index(drop=True)
mo = 0
de = 0
lst = []
lst_size = []
lst_major = []
lst_minor = []
lst_eccentricity = []
lst_orientation = []
lst_perimeter = []

for i in range(len(df)):
    df_temp = data_single[(data_single['subject'] == df.subject[i]) & (data_single['electrode1'] == df.electrode1[i]) & (data_single['freq'] == df.freq[i]) & (data_single['amp1'] == df.amp1[i])].reset_index(drop=True)
    ratio = len(df_temp[df_temp['num_regions']>1]) /len(df_temp)
    
    for row in range(len(df_temp)):
        df_temp['size'][row] = sum(df_temp['size'][row])
        df_temp['major_axis_length'][row] = sum(df_temp['major_axis_length'][row])
        df_temp['minor_axis_length'][row] = sum(df_temp['minor_axis_length'][row])
        df_temp['perimeter'][row] = sum(df_temp['perimeter'][row])
        df_temp['orientation_new'][row] = sum(df_temp['orientation_new'][row])
        
    
    count_length = len(df_temp)
    
    lst_size.append(df_temp['size'].sum()/count_length)
    lst_major.append(df_temp['major_axis_length'].sum()/count_length)
    lst_minor.append(df_temp['minor_axis_length'].sum()/count_length)
    lst_perimeter.append(df_temp['perimeter'].sum()/count_length)
    lst_orientation.append(df_temp['orientation_new'].sum()/count_length)
    
        
df['avg_size'] = lst_size
df['avg_major'] = lst_major
df['avg_minor'] = lst_minor
df['avg_perimeter'] = lst_perimeter
df['avg_orientation'] = lst_orientation


data_single = df.copy()

subject = ['12-005', '51-009', '52-001']
df = pd.DataFrame({})

for subj in subject:
    lst_e = []
    lst_dtf = []
    
    s2 = shapes.subject_params[subj]
    implant,model = shapes.model_from_params(s2)
    for i in string.ascii_uppercase[0:6]: 
        for j in range(1,11):
            electrode = i + str(j)
            lst_e.append(electrode)
            lst_dtf.append(math.sqrt(implant[electrode].x**2 +implant[electrode].y**2 ))
            
    df_o = pd.DataFrame(lst_e, columns=['electrode1'])
    df_o['distance_to_fovea'] = lst_dtf
    
    if subj == '12-005':
        lst_d = [0,0,0,2,7,7,0,0,5,0,
                0,0,4,11,13,15,15,3,7,4,
                0,0,15,16,17,17,19,16,9,4,
                0,0,16,19,15,17,22,25,13,10,
                0,0,8,15,14,13,17,23,14,2,
                0,0,10,15,13,12,9,11,5]
        lst_e = lst_e[:-1]
    elif subj == '51-009':
        lst_e = ['F1','F2','F3','F4','F5','F6',
               'E1','E2','E3','E4','E5','E6',
               'D1','D2','D3','D4','D5','D6','D7',
               'C1','C2','C3','C4','C5','C6','C7','C8',
               'B1','B2','B3','B4','B5','B6','B7','B8','B9','B10', 
               'A1','A2','A3','A4','A5','A6','A7','A8','A9','A10']
        lst_d = [0] * len(lst_e)
    else:
        lst_e = ['F1','F2','F4','F5','F6','F7','F8','F9','F10',
               'E1','E2','E3','E4','E5','E6','E7','E8','E9','E10', 
               'D1','D2','D3','D4','D5','D6','D7','D8','D9','D10', 
               'C3','C4','C5','C6','C7','C8','C9','C10', 
               'B1','B3','B4','B5','B6','B7','B8','B9','B10', 
               'A1','A2','A3','A4','A5','A6','A7','A8','A9','A10']
        lst_d = [0] * len(lst_e)
    lst_s = [subj] * len(lst_e)
    
    df_o = df_o[df_o.electrode1.isin(lst_e)]
    df_o['distance_to_implant'] = lst_d
    df_o['subject'] = lst_s
    df = pd.concat([df, df_o])
data_single = data_single.merge(df, how = 'inner', on = ['electrode1','subject'])


index_lst = []
for i in range(len(data_double)):
    if not (data_single[(data_single['subject'] == data_double.subject[i]) & (data_single['electrode1'] == data_double.electrode1[i]) & (data_single['freq'] == data_double.freq[i]) & (data_single['amp1'] == data_double.amp1[i])].empty):
        if not (data_single[(data_single['subject'] == data_double.subject[i]) & (data_single['electrode1'] == data_double.electrode2[i]) & (data_single['freq'] == data_double.freq[i]) & (data_single['amp1'] == data_double.amp1[i])].empty):
            index_lst.append(i)

### size

In [ ]:
df_AllSub = pd.DataFrame()

condition_list = [0]  
label_list = ['MultiElectrode']
for condition in condition_list:
    for label in range(1):
        data_double_temp = data_double.iloc[index_lst].reset_index(drop=True)
        data_double_temp = data_double_temp[data_double_temp['stim_class'] == label_list[label]]
        lst = data_double_temp[['electrode1','electrode2','amp1','freq','subject']]
        lst = lst.drop_duplicates().reset_index(drop=True)

        double = []
        single_dtf = []
        single_dti = []
        lst_subject = []
        
        distance = []
        amp = []
        freq = []
        electrode = []
        across = []
        along = []
        for i in range(len(lst)):
            electrode1 = lst.electrode1[i]
            electrode2 = lst.electrode2[i]
            temp1 = data_single[(data_single['electrode1'] == electrode1) &  (data_single['subject'] == lst.subject[i])  & (data_single['amp1'] == lst.amp1[i]) & (data_single['freq'] == lst.freq[i])].reset_index(drop=True)
            temp2 = data_single[(data_single['electrode1'] == electrode2) &  (data_single['subject'] == lst.subject[i])  & (data_single['amp1'] == lst.amp1[i]) & (data_single['freq'] == lst.freq[i])].reset_index(drop=True)
            double_temp = data_double_temp[(data_double_temp['electrode1'] == electrode1) & ((data_double_temp['electrode2'] == electrode2))& (data_double_temp['subject'] == lst.subject[i]) & (data_double_temp['amp1'] == lst.amp1[i]) & (data_double_temp['freq'] == lst.freq[i])].reset_index(drop=True)
            close_electrode = []
            
            if not temp1.empty and not temp2.empty and not double_temp.empty:
                if electrode1 not in close_electrode and electrode2 not in close_electrode:
                    
                    count = 0
                    for j in range(len(double_temp)):
                        count += sum(double_temp['size'][j])
                    double.append(count/len(double_temp))
                    single_dtf.append((temp1.distance_to_fovea[0] + temp2.distance_to_fovea[0])/2)
                    single_dti.append((temp1.distance_to_implant[0] + temp2.distance_to_implant[0])/2)
                    lst_subject.append(temp1.subject[0])

                    subject = shapes.subject_params[temp1.subject[0]]
                    implant,model = shapes.model_from_params(subject)

                    e1_x = implant[electrode1].x
                    e1_y = implant[electrode1].y
                    e2_x = implant[electrode2].x
                    e2_y = implant[electrode2].y

                    distance.append(math.sqrt((e1_x - e2_x)**2 + (e1_y - e2_y)**2))
                
                    amp.append(lst.amp1[i])
                    freq.append(lst.freq[i])

        df_investigate = pd.DataFrame({
                                       'double':double, 
                                       'freq':freq,
                                       'amp':amp,
                                       'single_dtf':single_dtf,
                                       'single_dti':single_dti,
            'distance':distance,
#                                         'across':across,
#                                         'along':along,
                                       'subject':lst_subject
                                      })
        
        df_AllSub = df_investigate.copy()
        
            
        temp = df_investigate[(df_investigate.amp == 2) & (df_investigate.freq == 20) & 
                              (df_investigate.single_dti == 0) &
                              (df_investigate.single_dtf > 2950) & (df_investigate.single_dtf < 3450)]
        
        temp = temp.groupby(['amp','freq','subject']).mean().reset_index()
        df = pd.DataFrame({})
        for subj in temp.subject:
            df_investigate_temp = df_investigate[df_investigate.subject == subj]
            df_investigate_temp['double'] = df_investigate_temp['double'] / temp[temp.subject == subj].iloc[0]['double']
            filename = subj + label_list[label]
            df_investigate_temp.to_csv('/home/yuchen/shapes/notebooks/' + filename + '.csv')
            df = pd.concat([df, df_investigate_temp])
        df_investigate = df.copy()
        filename = label_list[label]
        df_investigate.to_csv('/home/yuchen/shapes/notebooks/' + filename + '.csv')

In [ ]:
copy = df_investigate.copy()
df_AllSub=df_investigate.copy()

df_s = pd.DataFrame({})
for subj in ['12-005','51-009','52-001','All']:
    if subj=='All':
        df_t = df_AllSub.copy()
    else:   
        df_t = copy[copy.subject == subj]
    df_t['amp'] = (df_t['amp'] - df_t['amp'].mean()) / df_t['amp'].std()
    df_t['single_dti'] = (df_t['single_dti'] - df_t['single_dti'].mean()) / df_t['single_dti'].std()
    df_t['single_dtf'] = (df_t['single_dtf'] - df_t['single_dtf'].mean()) / df_t['single_dtf'].std()
    
    if subj == 'All':  
        df_AllSub = df_t.copy()
    else:
        df_s = pd.concat([df_s, df_t])
        df_s = df_s.fillna(0)
        
df_investigate = df_s.copy().reset_index(drop=True)

In [ ]:
fig, axs= plt.subplots(ncols=3, figsize=(21, 7))
column_lst = ['amp','single_dtf', 'single_dti']
name_lst = ['Amplitude','Electrode-Fovea Distance', 'Electrode-Retina Distance','Between-Axon Distance','Along-Axon Distance']
for dv in range(3):
    reg = LinearRegression().fit(df_AllSub[column_lst[:dv] + column_lst[dv+1 :]], df_AllSub[['double']])
    y_predicted = reg.predict(df_AllSub[column_lst[:dv] + column_lst[dv+1 :]])
    y = df_AllSub[['double']]-y_predicted

    reg = LinearRegression().fit(df_AllSub[column_lst[:dv] + column_lst[dv+1 :]], df_AllSub[column_lst[dv]])
    y_predicted = reg.predict(df_AllSub[column_lst[:dv] + column_lst[dv+1 :]])
    x = df_AllSub[column_lst[dv]]-y_predicted
    df_AllSub['x'] = x
    df_AllSub['y'] = y
    subject = ['12-005','51-009','52-001']
    marker_lst = ['o','s','^']
    color_lst = ['#1E88E5', '#FFC107','#004D40']
    for i in range(len(subject)):
        temp = df_AllSub[df_AllSub.subject == subject[i]]
        axs[dv].plot(temp['x'],temp['y'],marker_lst[i], color=color_lst[i],alpha=0.6,markersize=13)
    axs[dv].legend(subject, loc='upper right',prop={'size': 20})
    reg = LinearRegression().fit(np.array(x).reshape(-1,1),np.array(y).reshape(-1,1))
    y_pred = reg.predict(np.array(x).reshape(-1,1))
    axs[dv].plot(x, y_pred,'-', color="black",linewidth=2)
    axs[dv].set(xlabel = name_lst[dv],ylim=(-3.1,5.2), xlim=(-2,3.5))
    axs[0].set( ylabel = 'Area')
    axs[dv].tick_params(axis='both',  labelsize=18)
    
    axs[dv].get_xaxis().set_visible(False)
    if dv>0:
        axs[dv].get_yaxis().set_visible(False)
        
    axs[dv].yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
    
    for item in ([axs[dv].xaxis.label, axs[dv].yaxis.label]):
        item.set_fontsize(30)
fig.savefig('/home/yuchen/paper/12a. Double-Electrode Area.pdf', transparent=True)

In [ ]:
for subj in ['12-005','51-009','52-001','AllSubjects']:
    temp = df_investigate[df_investigate.subject == subj].reset_index(drop=True)
    if subj == '12-005':
        X = temp[['amp','single_dtf','single_dti']]
    elif subj == 'AllSubjects':
        temp = df_AllSub.copy()
        X = temp[['amp','single_dtf','single_dti']]
    else:
        X = temp[['amp','single_dtf']]
    y = temp.double
    X2 = sm.add_constant(X)
    est = sm.OLS(y, X2)
    est2 = est.fit()
    print('subject: ' + subj)
    print(est2.summary())
    print('\ncoefficient')
    [ print(round(x,5), end='\n') for x in est2.params.tolist()[1:]]
    print('\npvalues')
    [ print(round(x,5), end='\n') for x in est2.pvalues.tolist()[1:]]
    print('\nvariance inflation factor: ')
    print([[X.columns[i],variance_inflation_factor(X.values, i)]
                          for i in range(len(X.columns))])
    print('\npartial correlation: ')
    X['double'] = temp['double']
    [print(round(X.pcorr()['double'][i].tolist(), 5), end='\n') for i in X.pcorr()[:-1]]
    print(' ')
    

### major_axis_length

In [ ]:
df_AllSub = pd.DataFrame()

condition_list = [0]  
label_list = ['SpatialSummation','MultiElectrode']
label_list = ['MultiElectrode']
for condition in condition_list:
    for label in range(1):
        data_double_temp = data_double.iloc[index_lst].reset_index(drop=True)
        data_double_temp = data_double_temp[data_double_temp['stim_class'] == label_list[label]]
        lst = data_double_temp[['electrode1','electrode2','amp1','freq','subject']]
        lst = lst.drop_duplicates().reset_index(drop=True)

        double = []
        single_dtf = []
        single_dti = []
        lst_subject = []
        
        distance = []
        amp = []
        freq = []
        electrode = []
        across = []
        along = []
        for i in range(len(lst)):
            electrode1 = lst.electrode1[i]
            electrode2 = lst.electrode2[i]
            temp1 = data_single[(data_single['electrode1'] == electrode1) &  (data_single['subject'] == lst.subject[i])  & (data_single['amp1'] == lst.amp1[i]) & (data_single['freq'] == lst.freq[i])].reset_index(drop=True)
            temp2 = data_single[(data_single['electrode1'] == electrode2) &  (data_single['subject'] == lst.subject[i])  & (data_single['amp1'] == lst.amp1[i]) & (data_single['freq'] == lst.freq[i])].reset_index(drop=True)
            double_temp = data_double_temp[(data_double_temp['electrode1'] == electrode1) & ((data_double_temp['electrode2'] == electrode2))& (data_double_temp['subject'] == lst.subject[i]) & (data_double_temp['amp1'] == lst.amp1[i]) & (data_double_temp['freq'] == lst.freq[i])].reset_index(drop=True)
            close_electrode = []
            if not temp1.empty and not temp2.empty and not double_temp.empty:
                if electrode1 not in  and electrode2 not in close_electrode:
                    
                    count = 0
                    for j in range(len(double_temp)):
                        count += sum(double_temp['major_axis_length'][j])
                    double.append(count/len(double_temp))
                    single_dtf.append((temp1.distance_to_fovea[0] + temp2.distance_to_fovea[0])/2)
                    single_dti.append((temp1.distance_to_implant[0] + temp2.distance_to_implant[0])/2)
                    lst_subject.append(temp1.subject[0])

                    subject = shapes.subject_params[temp1.subject[0]]
                    implant,model = shapes.model_from_params(subject)

                    e1_x = implant[electrode1].x
                    e1_y = implant[electrode1].y
                    e2_x = implant[electrode2].x
                    e2_y = implant[electrode2].y

                    distance.append(math.sqrt((e1_x - e2_x)**2 + (e1_y - e2_y)**2))
                    

                    amp.append(lst.amp1[i])
                    freq.append(lst.freq[i])

        df_investigate = pd.DataFrame({
                                       'double':double, 
                                       'freq':freq,
                                       'amp':amp,
                                       'single_dtf':single_dtf,
                                       'single_dti':single_dti,
            'distance':distance,
                                       'subject':lst_subject
                                      })
        
        df_AllSub = df_investigate.copy()
        

        temp = df_investigate[(df_investigate.amp == 2) & (df_investigate.freq == 20) & 
                              (df_investigate.single_dti == 0) & 
                              (df_investigate.single_dtf > 2950) & (df_investigate.single_dtf < 3450)]
        temp = temp.groupby(['amp','freq','subject']).mean().reset_index()
        df = pd.DataFrame({})
        for subj in temp.subject:
            df_investigate_temp = df_investigate[df_investigate.subject == subj]
            df_investigate_temp['double'] = df_investigate_temp['double'] / temp[temp.subject == subj].iloc[0]['double']
            filename = subj + label_list[label]
            df_investigate_temp.to_csv('/home/yuchen/shapes/notebooks/' + filename + '.csv')
            df = pd.concat([df, df_investigate_temp])
        df_investigate = df.copy()
        filename = label_list[label]
        df_investigate.to_csv('/home/yuchen/shapes/notebooks/' + filename + '.csv')

In [ ]:
copy = df_investigate.copy()
df_AllSub = df_investigate.copy()

df_s = pd.DataFrame({})
for subj in ['12-005','51-009','52-001','All']:
    if subj=='All':
        df_t = df_AllSub.copy()
    else:   
        df_t = copy[copy.subject == subj]
    df_t['amp'] = (df_t['amp'] - df_t['amp'].mean()) / df_t['amp'].std()
    df_t['single_dti'] = (df_t['single_dti'] - df_t['single_dti'].mean()) / df_t['single_dti'].std()
    df_t['single_dtf'] = (df_t['single_dtf'] - df_t['single_dtf'].mean()) / df_t['single_dtf'].std()
    
    if subj == 'All':  
        df_AllSub = df_t.copy()
    else:
        df_s = pd.concat([df_s, df_t])
        df_s = df_s.fillna(0)
        
df_investigate = df_s.copy().reset_index(drop=True)

In [ ]:
fig, axs= plt.subplots(ncols=3, figsize=(21, 7))
column_lst = ['amp','single_dtf', 'single_dti']
name_lst = ['Amplitude','Foveal Eccentricity', 'Electrode-Implant Distance','Between-Axon Distance','Along-Axon Distance']
for dv in range(3):
    reg = LinearRegression().fit(df_AllSub[column_lst[:dv] + column_lst[dv+1 :]], df_AllSub[['double']])
    y_predicted = reg.predict(df_AllSub[column_lst[:dv] + column_lst[dv+1 :]])
    y = df_investigate[['double']]-y_predicted

    reg = LinearRegression().fit(df_AllSub[column_lst[:dv] + column_lst[dv+1 :]], df_AllSub[column_lst[dv]])
    y_predicted = reg.predict(df_AllSub[column_lst[:dv] + column_lst[dv+1 :]])
    x = df_AllSub[column_lst[dv]]-y_predicted
    df_AllSub['x'] = x
    df_AllSub['y'] = y
    subject = ['12-005','51-009','52-001']
    marker_lst = ['o','s','^']
    color_lst = ['#1E88E5', '#FFC107','#004D40']
    for i in range(len(subject)):
        temp = df_AllSub[df_AllSub.subject == subject[i]]
        axs[dv].plot(temp['x'],temp['y'],marker_lst[i], color=color_lst[i],alpha=0.6,markersize=13)
    axs[dv].legend(subject, loc='upper right',prop={'size': 20})
    reg = LinearRegression().fit(np.array(x).reshape(-1,1),np.array(y).reshape(-1,1))
    y_pred = reg.predict(np.array(x).reshape(-1,1))
    axs[dv].plot(x, y_pred,'-', color="black",linewidth=2)
#     axs[dv].set(xlabel = name_lst[dv], ylabel = 'Major Axis Length',ylim=(-0.7,0.8),xlim=(-2,3.4))
#     axs[dv].set(xlabel = name_lst[dv],ylim=(-0.75,0.8),xlim=(-2,3.5))
    axs[0].set(ylabel = 'Major Axis Length')
    axs[dv].tick_params(axis='both',  labelsize=18)
    
    axs[dv].get_xaxis().set_visible(False)
    if dv>0:
        axs[dv].get_yaxis().set_visible(False)
        
    axs[dv].yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
    
    for item in ([axs[dv].xaxis.label, axs[dv].yaxis.label]):
        item.set_fontsize(30)
fig.savefig('/home/yuchen/paper/12b. Double-Electrode Major Axis Length.pdf', transparent=True)

In [ ]:
for subj in ['12-005','51-009','52-001','AllSubjects']:
    temp = df_investigate[df_investigate.subject == subj].reset_index(drop=True)
    if subj == '12-005':
        X = temp[['amp','single_dtf','single_dti']]
    elif subj == 'AllSubjects':
        temp = df_AllSub.copy()
        X = temp[['amp','single_dtf','single_dti']]
    else:
        X = temp[['amp','single_dtf']]
    y = temp.double
    X2 = sm.add_constant(X)
    est = sm.OLS(y, X2)
    est2 = est.fit()
    print('subject: ' + subj)
    print(est2.summary())
    print('\ncoefficient')
    [ print(round(x,5), end='\n') for x in est2.params.tolist()[1:]]
    print('\npvalues')
    [ print(round(x,5), end='\n') for x in est2.pvalues.tolist()[1:]]
    print('\nvariance inflation factor: ')
    print([[X.columns[i],variance_inflation_factor(X.values, i)]
                          for i in range(len(X.columns))])
    print('\npartial correlation: ')
    X['double'] = temp['double']
    [print(round(X.pcorr()['double'][i].tolist(), 5), end='\n') for i in X.pcorr()[:-1]]
    print(' ')
    

### minor_axis_length

In [ ]:
df_AllSub = pd.DataFrame()

condition_list = [0]  
label_list = ['MultiElectrode']
for condition in condition_list:
    for label in range(1):
        data_double_temp = data_double.iloc[index_lst].reset_index(drop=True)
        data_double_temp = data_double_temp[data_double_temp['stim_class'] == label_list[label]]
        lst = data_double_temp[['electrode1','electrode2','amp1','freq','subject']]
        lst = lst.drop_duplicates().reset_index(drop=True)

        double = []
        single_dtf = []
        single_dti = []
        lst_subject = []
        
        distance = []
        amp = []
        freq = []
        electrode = []
        across = []
        along = []
        for i in range(len(lst)):
            electrode1 = lst.electrode1[i]
            electrode2 = lst.electrode2[i]
            temp1 = data_single[(data_single['electrode1'] == electrode1) &  (data_single['subject'] == lst.subject[i])  & (data_single['amp1'] == lst.amp1[i]) & (data_single['freq'] == lst.freq[i])].reset_index(drop=True)
            temp2 = data_single[(data_single['electrode1'] == electrode2) &  (data_single['subject'] == lst.subject[i])  & (data_single['amp1'] == lst.amp1[i]) & (data_single['freq'] == lst.freq[i])].reset_index(drop=True)
            double_temp = data_double_temp[(data_double_temp['electrode1'] == electrode1) & ((data_double_temp['electrode2'] == electrode2))& (data_double_temp['subject'] == lst.subject[i]) & (data_double_temp['amp1'] == lst.amp1[i]) & (data_double_temp['freq'] == lst.freq[i])].reset_index(drop=True)
            close_electrode = []
            if not temp1.empty and not temp2.empty and not double_temp.empty:
                if electrode1 not in close_electrode and electrode2 not in close_electrode:
                    
                    count = 0
                    for j in range(len(double_temp)):
                        count += sum(double_temp['minor_axis_length'][j])
                    double.append(count/len(double_temp))
                    single_dtf.append((temp1.distance_to_fovea[0] + temp2.distance_to_fovea[0])/2)
                    single_dti.append((temp1.distance_to_implant[0] + temp2.distance_to_implant[0])/2)
                    lst_subject.append(temp1.subject[0])

                    subject = shapes.subject_params[temp1.subject[0]]
                    implant,model = shapes.model_from_params(subject)

                    e1_x = implant[electrode1].x
                    e1_y = implant[electrode1].y
                    e2_x = implant[electrode2].x
                    e2_y = implant[electrode2].y

                    distance.append(math.sqrt((e1_x - e2_x)**2 + (e1_y - e2_y)**2))

                    amp.append(lst.amp1[i])
                    freq.append(lst.freq[i])

        df_investigate = pd.DataFrame({
                                       'double':double, 
                                       'freq':freq,
                                       'amp':amp,
                                       'single_dtf':single_dtf,
                                       'single_dti':single_dti,
            'distance':distance,
                                       'subject':lst_subject
                                      })

        temp = df_investigate[(df_investigate.amp == 2) & (df_investigate.freq == 20) & 
                              (df_investigate.single_dti == 0) & 
                              (df_investigate.single_dtf > 2950) & (df_investigate.single_dtf < 3450)]
        temp = temp.groupby(['amp','freq','subject']).mean().reset_index()
        df = pd.DataFrame({})
        for subj in temp.subject:
            df_investigate_temp = df_investigate[df_investigate.subject == subj]
            df_investigate_temp['double'] = df_investigate_temp['double'] / temp[temp.subject == subj].iloc[0]['double']
            filename = subj + label_list[label]
            df_investigate_temp.to_csv('/home/yuchen/shapes/notebooks/' + filename + '.csv')
            df = pd.concat([df, df_investigate_temp])
        df_investigate = df.copy()
        filename = label_list[label]
        df_investigate.to_csv('/home/yuchen/shapes/notebooks/' + filename + '.csv')

In [ ]:
copy = df_investigate.copy()
df_AllSub = df_investigate.copy()

df_s = pd.DataFrame({})
for subj in ['12-005','51-009','52-001','All']:
    if subj=='All':
        df_t = df_AllSub.copy()
    else:   
        df_t = copy[copy.subject == subj]
    df_t['amp'] = (df_t['amp'] - df_t['amp'].mean()) / df_t['amp'].std()
    df_t['single_dti'] = (df_t['single_dti'] - df_t['single_dti'].mean()) / df_t['single_dti'].std()
    df_t['single_dtf'] = (df_t['single_dtf'] - df_t['single_dtf'].mean()) / df_t['single_dtf'].std()

    if subj == 'All':  
        df_AllSub = df_t.copy()
    else:
        df_s = pd.concat([df_s, df_t])
        df_s = df_s.fillna(0)
        
df_investigate = df_s.copy().reset_index(drop=True)

In [ ]:
fig, axs= plt.subplots(ncols=3, figsize=(21, 7))
column_lst = ['amp','single_dtf', 'single_dti']
name_lst = ['Amplitude','Electrode-Fovea Distance', 'Electrode-Retina Distance','Between-Axon Distance','Along-Axon Distance']
for dv in range(3):
    reg = LinearRegression().fit(df_AllSub[column_lst[:dv] + column_lst[dv+1 :]], df_AllSub[['double']])
    y_predicted = reg.predict(df_AllSub[column_lst[:dv] + column_lst[dv+1 :]])
    y = df_AllSub[['double']]-y_predicted

    reg = LinearRegression().fit(df_AllSub[column_lst[:dv] + column_lst[dv+1 :]], df_AllSub[column_lst[dv]])
    y_predicted = reg.predict(df_AllSub[column_lst[:dv] + column_lst[dv+1 :]])
    x = df_AllSub[column_lst[dv]]-y_predicted
    df_AllSub['x'] = x
    df_AllSub['y'] = y
    subject = ['12-005','51-009','52-001']
    marker_lst = ['o','s','^']
    color_lst = ['#1E88E5', '#FFC107','#004D40']
    for i in range(len(subject)):
        temp = df_AllSub[df_AllSub.subject == subject[i]]
        axs[dv].plot(temp['x'],temp['y'],marker_lst[i], color=color_lst[i],alpha=0.6,markersize=13)
    axs[dv].legend(subject, loc='upper right',prop={'size': 20})
    reg = LinearRegression().fit(np.array(x).reshape(-1,1),np.array(y).reshape(-1,1))
    y_pred = reg.predict(np.array(x).reshape(-1,1))
    axs[dv].plot(x, y_pred,'-', color="black",linewidth=2)
    axs[dv].set(xlabel = name_lst[dv],ylim=(-1.3,2.1),xlim=(-2,3.5))
    axs[0].set(ylabel = 'Minor Axis Length')
    axs[dv].tick_params(axis='both',  labelsize=18)

    if dv>0:
        axs[dv].get_yaxis().set_visible(False)
    axs[dv].yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
        
    for item in ([axs[dv].xaxis.label, axs[dv].yaxis.label]):
        item.set_fontsize(30)
fig.savefig('/home/yuchen/paper/12c. Double-Electrode Minor Axis Length.pdf', transparent=True)

In [ ]:
for subj in ['12-005','51-009','52-001','AllSubjects']:
    temp = df_investigate[df_investigate.subject == subj].reset_index(drop=True)
    if subj == '12-005':
        X = temp[['amp','single_dtf','single_dti']]
    elif subj == 'AllSubjects':
        temp = df_AllSub.copy()
        X = temp[['amp','single_dtf','single_dti']]
    else:
        X = temp[['amp','single_dtf']]
    y = temp.double
    X2 = sm.add_constant(X)
    est = sm.OLS(y, X2)
    est2 = est.fit()
    print('subject: ' + subj)
    print(est2.summary())
    print('\ncoefficient')
    [ print(round(x,5), end='\n') for x in est2.params.tolist()[1:]]
    print('\npvalues')
    [ print(round(x,5), end='\n') for x in est2.pvalues.tolist()[1:]]
    print('\nvariance inflation factor: ')
    print([[X.columns[i],variance_inflation_factor(X.values, i)]
                          for i in range(len(X.columns))])
    print('\npartial correlation: ')
    X['double'] = temp['double']
    [print(round(X.pcorr()['double'][i].tolist(), 5), end='\n') for i in X.pcorr()[:-1]]
    print(' ')
    

### perimeter

In [ ]:
df_AllSub = pd.DataFrame()

condition_list = [0]  
label_list = ['MultiElectrode']
for condition in condition_list:
    for label in range(1):
        data_double_temp = data_double.iloc[index_lst].reset_index(drop=True)
        data_double_temp = data_double_temp[data_double_temp['stim_class'] == label_list[label]]
        lst = data_double_temp[['electrode1','electrode2','amp1','freq','subject']]
        lst = lst.drop_duplicates().reset_index(drop=True)

        double = []
        single_dtf = []
        single_dti = []
        lst_subject = []
        
        distance = []
        amp = []
        freq = []
        electrode = []
        across = []
        along = []
        for i in range(len(lst)):
            electrode1 = lst.electrode1[i]
            electrode2 = lst.electrode2[i]
            temp1 = data_single[(data_single['electrode1'] == electrode1) &  (data_single['subject'] == lst.subject[i])  & (data_single['amp1'] == lst.amp1[i]) & (data_single['freq'] == lst.freq[i])].reset_index(drop=True)
            temp2 = data_single[(data_single['electrode1'] == electrode2) &  (data_single['subject'] == lst.subject[i])  & (data_single['amp1'] == lst.amp1[i]) & (data_single['freq'] == lst.freq[i])].reset_index(drop=True)
            double_temp = data_double_temp[(data_double_temp['electrode1'] == electrode1) & ((data_double_temp['electrode2'] == electrode2))& (data_double_temp['subject'] == lst.subject[i]) & (data_double_temp['amp1'] == lst.amp1[i]) & (data_double_temp['freq'] == lst.freq[i])].reset_index(drop=True)
            close_electrode = []
            
            if not temp1.empty and not temp2.empty and not double_temp.empty:
                if electrode1 not in close_electrode and electrode2 not in close_electrode:
                    
                    count = 0
                    for j in range(len(double_temp)):
                        count += sum(double_temp['perimeter'][j])
                    double.append(count/len(double_temp))
                    single_dtf.append((temp1.distance_to_fovea[0] + temp2.distance_to_fovea[0])/2)
                    single_dti.append((temp1.distance_to_implant[0] + temp2.distance_to_implant[0])/2)
                    lst_subject.append(temp1.subject[0])

                    subject = shapes.subject_params[temp1.subject[0]]
                    implant,model = shapes.model_from_params(subject)
                    
                    e1_x = implant[electrode1].x
                    e1_y = implant[electrode1].y
                    e2_x = implant[electrode2].x
                    e2_y = implant[electrode2].y

                    distance.append(math.sqrt((e1_x - e2_x)**2 + (e1_y - e2_y)**2))

                    amp.append(lst.amp1[i])
                    freq.append(lst.freq[i])

        df_investigate = pd.DataFrame({
                                       'double':double, 
                                       'freq':freq,
                                       'amp':amp,
                                       'single_dtf':single_dtf,
                                       'single_dti':single_dti,
            'distance':distance,
                                       'subject':lst_subject
                                      })
        
        temp = df_investigate[(df_investigate.amp == 2) & (df_investigate.freq == 20) & 
                              (df_investigate.single_dti == 0) & 
                              (df_investigate.single_dtf > 2950) & (df_investigate.single_dtf < 3450)]
        temp = temp.groupby(['amp','freq','subject']).mean().reset_index()
        df = pd.DataFrame({})
        for subj in temp.subject:
            df_investigate_temp = df_investigate[df_investigate.subject == subj]
            df_investigate_temp['double'] = df_investigate_temp['double'] / temp[temp.subject == subj].iloc[0]['double']
            filename = subj + label_list[label]
            df_investigate_temp.to_csv('/home/yuchen/shapes/notebooks/' + filename + '.csv')
            df = pd.concat([df, df_investigate_temp])
        df_investigate = df.copy()
        filename = label_list[label]
        df_investigate.to_csv('/home/yuchen/shapes/notebooks/' + filename + '.csv')

In [ ]:
copy = df_investigate.copy()
df_AllSub = df_investigate.copy()

df_s = pd.DataFrame({})
for subj in ['12-005','51-009','52-001','All']:
    if subj=='All':
        df_t = df_AllSub.copy()
    else:   
        df_t = copy[copy.subject == subj]
    df_t['amp'] = (df_t['amp'] - df_t['amp'].mean()) / df_t['amp'].std()
    df_t['single_dti'] = (df_t['single_dti'] - df_t['single_dti'].mean()) / df_t['single_dti'].std()
    df_t['single_dtf'] = (df_t['single_dtf'] - df_t['single_dtf'].mean()) / df_t['single_dtf'].std()
    
    if subj == 'All':  
        df_AllSub = df_t.copy()
    else:
        df_s = pd.concat([df_s, df_t])
        df_s = df_s.fillna(0)
        
df_investigate = df_s.copy().reset_index(drop=True)

In [ ]:
fig, axs= plt.subplots(ncols=3, figsize=(21, 7))
column_lst = ['amp','single_dtf', 'single_dti']
name_lst = ['Amplitude','Foveal Eccentricity', 'Electrode-Implant Distance','Between-Axon Distance','Along-Axon Distance']
for dv in range(3):
    reg = LinearRegression().fit(df_AllSub[column_lst[:dv] + column_lst[dv+1 :]], df_AllSub[['double']])
    y_predicted = reg.predict(df_AllSub[column_lst[:dv] + column_lst[dv+1 :]])
    y = df_AllSub[['double']]-y_predicted

    reg = LinearRegression().fit(df_AllSub[column_lst[:dv] + column_lst[dv+1 :]], df_AllSub[column_lst[dv]])
    y_predicted = reg.predict(df_AllSub[column_lst[:dv] + column_lst[dv+1 :]])
    x = df_AllSub[column_lst[dv]]-y_predicted
    df_AllSub['x'] = x
    df_AllSub['y'] = y
    subject = ['12-005','51-009','52-001']
    marker_lst = ['o','s','^']
    color_lst = ['#1E88E5', '#FFC107','#004D40']
    for i in range(len(subject)):
        temp = df_AllSub[df_AllSub.subject == subject[i]]
        axs[dv].plot(temp['x'],temp['y'],marker_lst[i], color=color_lst[i],alpha=0.6,markersize=13)
    axs[dv].legend(subject, loc='upper right',prop={'size': 20})
    reg = LinearRegression().fit(np.array(x).reshape(-1,1),np.array(y).reshape(-1,1))
    y_pred = reg.predict(np.array(x).reshape(-1,1))
    axs[dv].plot(x, y_pred,'-', color="black",linewidth=2)
    axs[dv].set(xlabel = name_lst[dv],ylim=(-0.75,0.7),xlim=(-2,3.5))
    axs[0].set(ylabel = 'Perimeter')
    axs[dv].tick_params(axis='both',  labelsize=18)
    
    axs[dv].get_xaxis().set_visible(False)
    if dv>0:
        axs[dv].get_yaxis().set_visible(False)
    axs[dv].yaxis.set_major_formatter(FormatStrFormatter('%.1f'))

    for item in ([axs[dv].xaxis.label, axs[dv].yaxis.label]):
        item.set_fontsize(30)
fig.savefig('/home/yuchen/paper/12d. Double-Electrode Perimeter.pdf', transparent=True)

In [ ]:
for subj in ['12-005','51-009','52-001','AllSubjects']:
    temp = df_investigate[df_investigate.subject == subj].reset_index(drop=True)
    if subj == '12-005':
        X = temp[['amp','single_dtf','single_dti']]
    elif subj == 'AllSubjects':
        temp = df_AllSub.copy()
        X = temp[['amp','single_dtf','single_dti']]
    else:
        X = temp[['amp','single_dtf']]
    y = temp.double
    X2 = sm.add_constant(X)
    est = sm.OLS(y, X2)
    est2 = est.fit()
    print('subject: ' + subj)
    print(est2.summary())
    print('\ncoefficient')
    [ print(round(x,5), end='\n') for x in est2.params.tolist()[1:]]
    print('\npvalues')
    [ print(round(x,5), end='\n') for x in est2.pvalues.tolist()[1:]]
    print('\nvariance inflation factor: ')
    print([[X.columns[i],variance_inflation_factor(X.values, i)]
                          for i in range(len(X.columns))])
    print('\npartial correlation: ')
    X['double'] = temp['double']
    [print(round(X.pcorr()['double'][i].tolist(), 5), end='\n') for i in X.pcorr()[:-1]]
    print(' ')
    

### orientation

In [ ]:
df_AllSub = pd.DataFrame()

condition_list = [0]  
label_list = ['MultiElectrode']
for condition in condition_list:
    for label in range(1):
        data_double_temp = data_double.iloc[index_lst].reset_index(drop=True)
        data_double_temp = data_double_temp[data_double_temp['stim_class'] == label_list[label]]
        lst = data_double_temp[['electrode1','electrode2','amp1','freq','subject']]
        lst = lst.drop_duplicates().reset_index(drop=True)

        double = []
        single_dtf = []
        single_dti = []
        lst_subject = []
        between = []
        along = []
        
        distance = []
        amp = []
        freq = []
        tan = []
        electrode = []
        for i in range(len(lst)):
            electrode1 = lst.electrode1[i]
            electrode2 = lst.electrode2[i]
            temp1 = data_single[(data_single['electrode1'] == electrode1) &  (data_single['subject'] == lst.subject[i])  & (data_single['amp1'] == lst.amp1[i]) & (data_single['freq'] == lst.freq[i])].reset_index(drop=True)
            temp2 = data_single[(data_single['electrode1'] == electrode2) &  (data_single['subject'] == lst.subject[i])  & (data_single['amp1'] == lst.amp1[i]) & (data_single['freq'] == lst.freq[i])].reset_index(drop=True)
            double_temp = data_double_temp[(data_double_temp['electrode1'] == electrode1) & ((data_double_temp['electrode2'] == electrode2))& (data_double_temp['subject'] == lst.subject[i]) & (data_double_temp['amp1'] == lst.amp1[i]) & (data_double_temp['freq'] == lst.freq[i])].reset_index(drop=True)
            if not temp1.empty and not temp2.empty and not double_temp.empty:
                count = 0
                for j in range(len(double_temp)):
                    count += mean(double_temp['orientation_new'][j])
                double.append(count/len(double_temp))
                single_dtf.append((temp1.distance_to_fovea[0] + temp2.distance_to_fovea[0])/2)
                single_dti.append((temp1.distance_to_implant[0] + temp2.distance_to_implant[0])/2)
                lst_subject.append(temp1.subject[0])
                
                subject = shapes.subject_params[temp1.subject[0]]
                implant,model = shapes.model_from_params(subject)
                
                e1_x = implant[electrode1].x
                e1_y = implant[electrode1].y
                e2_x = implant[electrode2].x
                e2_y = implant[electrode2].y
                
                tan.append(mean([model.calc_bundle_tangent(e1_x, e1_y)*-1, model.calc_bundle_tangent(e2_x, e2_y)*-1]))
                
                distance.append(math.sqrt((e1_x - e2_x)**2 + (e1_y - e2_y)**2))

                amp.append(lst.amp1[i])
                freq.append(lst.freq[i])

        df_investigate = pd.DataFrame({
                                       'double':double, 
                                       'freq':freq,
                                       'amp':amp,
                                        'distance':distance,
            'tan':tan,
            
                                       'single_dtf':single_dtf,
                                       'single_dti':single_dti,
                                       'subject':lst_subject
                                      })
        for subj in ['12-005', '51-009', '52-001']:
            df_investigate_temp = df_investigate[df_investigate.subject == subj]
            filename = subj + label_list[label]
            df_investigate_temp.to_csv('/home/yuchen/shapes/notebooks/' + filename + '.csv')
        filename = label_list[label]
        df_investigate.to_csv('/home/yuchen/shapes/notebooks/' + filename + '.csv')
        
df_AllSub = df_investigate.copy()

In [ ]:
for i in range(len(df_investigate)):
    if df_investigate.tan[i] < 0:
        df_investigate.at[i,'tan'] = df_investigate.tan[i]+np.pi
    if df_investigate.double[i] < 0:
        df_investigate.at[i,'double'] = df_investigate.double[i]+np.pi

In [ ]:
copy = df_investigate.copy()
df_AllSub = df_investigate.copy()

df_s = pd.DataFrame({})
for subj in ['12-005','51-009','52-001','All']:
    if subj=='All':
        df_t = df_AllSub.copy()
    else:   
        df_t = copy[copy.subject == subj]
    df_t['amp'] = (df_t['amp'] - df_t['amp'].mean()) / df_t['amp'].std()
    df_t['single_dti'] = (df_t['single_dti'] - df_t['single_dti'].mean()) / df_t['single_dti'].std()
    df_t['single_dtf'] = (df_t['single_dtf'] - df_t['single_dtf'].mean()) / df_t['single_dtf'].std()
    
    if subj == 'All':  
        df_AllSub = df_t.copy()
    else:
        df_s = pd.concat([df_s, df_t])
        df_s = df_s.fillna(0)
        
df_investigate = df_s.copy().reset_index(drop=True)

In [ ]:
fig, axs= plt.subplots(ncols=6, figsize=(42, 7))
column_lst = ['amp','single_dtf', 'single_dti','between','along','tan']
name_lst = ['Amplitude','Foveal Eccentricity', 'Electrode-Implant Distance','Between-Axon Distance','Along-Axon Distance', 'Axonal Tangent Line']
for dv in range(6):
    reg = LinearRegression().fit(df_investigate[column_lst[:dv] + column_lst[dv+1 :]], df_investigate[['double']])
    y_predicted = reg.predict(df_investigate[column_lst[:dv] + column_lst[dv+1 :]])
    y = df_investigate[['double']]-y_predicted

    reg = LinearRegression().fit(df_investigate[column_lst[:dv] + column_lst[dv+1 :]], df_investigate[column_lst[dv]])
    y_predicted = reg.predict(df_investigate[column_lst[:dv] + column_lst[dv+1 :]])
    x = df_investigate[column_lst[dv]]-y_predicted
    df_investigate['x'] = x
    df_investigate['y'] = y
    subject = ['12-005','51-009','52-001']
    marker_lst = ['o','s','^']
    color_lst = ['#1E88E5', '#FFC107','#004D40']
    for i in range(len(subject)):
        temp = df_investigate[df_investigate.subject == subject[i]]
        axs[dv].plot(temp['x'],temp['y'],marker_lst[i], color=color_lst[i],alpha=0.6,markersize=13)
    axs[dv].legend(subject, loc='upper right',prop={'size': 15})
    reg = LinearRegression().fit(np.array(x).reshape(-1,1),np.array(y).reshape(-1,1))
    y_pred = reg.predict(np.array(x).reshape(-1,1))
    axs[dv].plot(x, y_pred,'-', color="black",linewidth=2)
    axs[dv].set(xlabel = name_lst[dv],ylim=(-2.6,2.6),xlim=(-2.2,3.5))
    axs[0].set(ylabel = 'Orientation')
    axs[dv].tick_params(axis='both',  labelsize=18)
    
    for item in ([axs[dv].xaxis.label, axs[dv].yaxis.label]):
        item.set_fontsize(30)
fig.savefig('/home/yuchen/paper/12e. Double-Electrode Orientation.pdf', transparent=True)

In [ ]:
for subj in ['12-005','51-009','52-001','AllSubjects']:
    temp = df_investigate[df_investigate.subject == subj].reset_index(drop=True)
    if subj == '12-005':
        X = temp[['amp','single_dtf','single_dti','tan']]
    elif subj == 'AllSubjects':
        temp = df_AllSub.copy()
        X = temp[['amp','single_dtf','single_dti','tan']]
    else:
        X = temp[['amp','single_dtf','tan']]
    y = temp.double
    X2 = sm.add_constant(X)
    est = sm.OLS(y, X2)
    est2 = est.fit()
    print('subject: ' + subj)
    print(est2.summary())
    print('\ncoefficient with more digits: ')
    print(est2.params)
    print('\nvariance inflation factor: ')
    print([[X.columns[i],variance_inflation_factor(X.values, i)]
                          for i in range(len(X.columns))])
    print('\npartial correlation: ')
    print(temp[['amp','single_dtf','single_dti','tan','double']].pcorr())
    print(' ')

### number of phosphene

In [ ]:
data = data_copy.copy()

data_double = data[(data['electrode2'] != str())].reset_index(drop=True)
data_single = data[(data['electrode2'] == str())].reset_index(drop=True)
data_single['stim_class'] ='SingleElectrode'
df = data_single[['subject', 'electrode1', 'freq', 'amp1']].drop_duplicates().reset_index(drop=True)
lst = []
lst_phosphene = []


for i in range(len(df)):
    df_temp = data_single[(data_single['subject'] == df.subject[i]) & (data_single['electrode1'] == df.electrode1[i]) & (data_single['freq'] == df.freq[i]) & (data_single['amp1'] == df.amp1[i])].reset_index(drop=True)
    ratio = len(df_temp[df_temp['num_regions']>1]) /len(df_temp)
    lst.append(ratio)
    lst_phosphene.append(mean(df_temp['num_regions']))
    

df['phosphene'] = lst_phosphene
data_single = df

subject = ['12-005', '51-009', '52-001']
df = pd.DataFrame({})

for subj in subject:
    lst_e = []
    lst_dtf = []
    
    s2 = shapes.subject_params[subj]
    implant,model = shapes.model_from_params(s2)
    for i in string.ascii_uppercase[0:6]: 
        for j in range(1,11):
            electrode = i + str(j)
            lst_e.append(electrode)
            lst_dtf.append(math.sqrt(implant[electrode].x**2 +implant[electrode].y**2 ))
    df_o = pd.DataFrame(lst_e, columns=['electrode1'])
    df_o['distance_to_fovea'] = lst_dtf
    
    if subj == '12-005':
        lst_d = [0,0,0,2,7,7,0,0,5,0,
                0,0,4,11,13,15,15,3,7,4,
                0,0,15,16,17,17,19,16,9,4,
                0,0,16,19,15,17,22,25,13,10,
                0,0,8,15,14,13,17,23,14,2,
                0,0,10,15,13,12,9,11,5]
        lst_e = lst_e[:-1]
    elif subj == '51-009':
        lst_e = ['F1','F2','F3','F4','F5','F6',
               'E1','E2','E3','E4','E5','E6',
               'D1','D2','D3','D4','D5','D6','D7',
               'C1','C2','C3','C4','C5','C6','C7','C8',
               'B1','B2','B3','B4','B5','B6','B7','B8','B9','B10', 
               'A1','A2','A3','A4','A5','A6','A7','A8','A9','A10']
        lst_d = [0] * len(lst_e)
    else:
        lst_e = ['F1','F2','F4','F5','F6','F7','F8','F9','F10',
               'E1','E2','E3','E4','E5','E6','E7','E8','E9','E10', 
               'D1','D2','D3','D4','D5','D6','D7','D8','D9','D10', 
               'C3','C4','C5','C6','C7','C8','C9','C10', 
               'B1','B3','B4','B5','B6','B7','B8','B9','B10', 
               'A1','A2','A3','A4','A5','A6','A7','A8','A9','A10']
        lst_d = [0] * len(lst_e)
    lst_s = [subj] * len(lst_e)
    
    df_o = df_o[df_o.electrode1.isin(lst_e)]
    
    df_o['distance_to_implant'] = lst_d
    df_o['subject'] = lst_s
    
    df = pd.concat([df, df_o])
data_single = data_single.merge(df, how = 'inner', on = ['electrode1','subject'])


index_lst = []
for i in range(len(data_double)):
    if not (data_single[(data_single['subject'] == data_double.subject[i]) & (data_single['electrode1'] == data_double.electrode1[i]) & (data_single['freq'] == data_double.freq[i]) & (data_single['amp1'] == data_double.amp1[i])].empty):
        if not (data_single[(data_single['subject'] == data_double.subject[i]) & (data_single['electrode1'] == data_double.electrode2[i]) & (data_single['freq'] == data_double.freq[i]) & (data_single['amp1'] == data_double.amp1[i])].empty):
            index_lst.append(i)

In [ ]:
%cd /home/yuchen/shapes/shapes

In [ ]:
from distances import dist_across_along, dist_perpendicular_tangential

In [ ]:
subj_params = {
    '12-005': {
        'subject_id': 'S2',
        'implant_type_str': 'ArgusII',
        'implant_x': -1896,
        'implant_y': -542,
        'implant_rot': -44,
        'loc_od_x': 15.8,
        'loc_od_y': 1.4,
        'xmin': -30,
        'xmax': 30,
        'ymin': -22.5,
        'ymax': 22.5
    },
    '51-009': {
        'subject_id': 'S3',
        'implant_type_str': 'ArgusII',
        'implant_x': -1203,
        'implant_y': 280,
        'implant_rot': -35,
        'loc_od_x': 15.4,
        'loc_od_y': 1.57,
        'xmin': -32.5,
        'xmax': 32.5,
        'ymin': -24.4,
        'ymax': 24.4
    },
    '52-001': {
        'subject_id': 'S4',
        'implant_type_str': 'ArgusII',
        'implant_x': -1945,
        'implant_y': 469,
        'implant_rot': -34,
        'loc_od_x': 15.8,
        'loc_od_y': 1.51,
        'xmin': -32,
        'xmax': 32,
        'ymin': -24,
        'ymax': 24
    }}

models = {
    '12-005': p2p.models.AxonMapModel(loc_od=(subj_params['12-005']['loc_od_x'], subj_params['12-005']['loc_od_y'])),
    '51-009': p2p.models.AxonMapModel(loc_od=(subj_params['51-009']['loc_od_x'], subj_params['51-009']['loc_od_y'])),
    '52-001': p2p.models.AxonMapModel(loc_od=(subj_params['52-001']['loc_od_x'], subj_params['52-001']['loc_od_y'])),
}
implants = {
    '12-005': p2p.implants.ArgusII(x=subj_params['12-005']['implant_x'], y=subj_params['12-005']['implant_y'], 
                                   rot=subj_params['12-005']['implant_rot']),
    '51-009': p2p.implants.ArgusII(x=subj_params['51-009']['implant_x'], y=subj_params['51-009']['implant_y'], 
                                   rot=subj_params['51-009']['implant_rot']),
    '52-001': p2p.implants.ArgusII(x=subj_params['52-001']['implant_x'], y=subj_params['52-001']['implant_y'], 
                                   rot=subj_params['52-001']['implant_rot'])
}

def too_close(subject, delta_y=200, delta_rot=10):
    """Which electrodes would fall on the other side of the raphe if y or rot were a little off?"""
    # y value within abs(delta_y)? This means shifting up/down will push them
    # on the other side:
    implant = implants[subject]
    electrodes = set()
    if delta_y > 0:
        electrodes = set([n for n, e in implant.electrodes.items() if np.abs(e.y) < delta_y])
    if delta_rot > 0:
        for rot in [-delta_rot, delta_rot]:
            new_implant = p2p.implants.ArgusII(x=subj_params[subject]['implant_x'], 
                                            y=subj_params[subject]['implant_y'], 
                                            rot=subj_params[subject]['implant_rot'] + rot)
            # if signbit is true, the y value changed sign by rotating the implant:
            changed = set([n for n, e in new_implant.electrodes.items() if np.signbit(e.y * implant[n].y)])
            electrodes = electrodes.union(changed)
    return list(electrodes)


In [ ]:
label_list = ['MultiElectrode']
df_AllSub = pd.DataFrame()

for label in range(1):
    data_double_temp = data_double.iloc[index_lst].reset_index(drop=True)
    data_double_temp = data_double_temp[data_double_temp['stim_class'] == label_list[label]]
    lst = data_double_temp[['electrode1','electrode2','amp1','freq','subject']]
    lst = lst.drop_duplicates().reset_index(drop=True)

    double = []
    single_dtf = []
    single_dti = []
    lst_subject = []
        
    distance = []
    amp = []
    freq = []
    between = []
    along_1 = []
    along_2 = []
    between_1 = []
    between_2 = []
    side = []
    
    across = []
    along = []
    electrode = []
    electrode_pair = []
    for i in range(len(lst)):
        electrode1 = lst.electrode1[i]
        electrode2 = lst.electrode2[i]
        temp1 = data_single[(data_single['electrode1'] == electrode1) &  (data_single['subject'] == lst.subject[i])  & (data_single['amp1'] == lst.amp1[i]) & (data_single['freq'] == lst.freq[i])].reset_index(drop=True)
        temp2 = data_single[(data_single['electrode1'] == electrode2) &  (data_single['subject'] == lst.subject[i])  & (data_single['amp1'] == lst.amp1[i]) & (data_single['freq'] == lst.freq[i])].reset_index(drop=True)
        double_temp = data_double_temp[(data_double_temp['electrode1'] == electrode1) & ((data_double_temp['electrode2'] == electrode2))& (data_double_temp['subject'] == lst.subject[i]) & (data_double_temp['amp1'] == lst.amp1[i]) & (data_double_temp['freq'] == lst.freq[i])].reset_index(drop=True)
        close_electrode = too_close(temp1.subject[0], delta_y=100, delta_rot=5)
        if not temp1.empty and not temp2.empty and not double_temp.empty:
            if electrode1 not in close_electrode and electrode2 not in close_electrode:
                count = 0
                for j in range(len(double_temp)):
                    count += double_temp['num_regions'][j]
                double.append(count/len(double_temp))
                single_dtf.append((temp1.distance_to_fovea[0] + temp2.distance_to_fovea[0])/2)
                single_dti.append((temp1.distance_to_implant[0] + temp2.distance_to_implant[0])/2)
                lst_subject.append(temp1.subject[0])

                subject = shapes.subject_params[temp1.subject[0]]
                implant,model = shapes.model_from_params(subject)

                arr = dist_across_along([electrode1], [electrode2], implant, model,strategy='radial')
                across.append(arr[0,1])
                along.append(arr[0,2])

                electrode_pair.append(electrode1 + '_' + electrode2)
                amp.append(lst.amp1[i])
                freq.append(lst.freq[i])

        df_investigate = pd.DataFrame({
                                           'double':double, 
                                           'freq':freq,
                                           'amp':amp,
                                            'across':across,
                                            'along':along,
                                            'electrode_pair':electrode_pair,
                                           'single_dtf':single_dtf,
                                           'single_dti':single_dti,
                                           'subject':lst_subject
                                          })
        for subj in ['12-005', '51-009', '52-001']:
            df_investigate_temp = df_investigate[df_investigate.subject == subj]
            filename = subj + label_list[label]
            df_investigate_temp.to_csv('/home/yuchen/shapes/notebooks/' + filename + '.csv')
        filename = label_list[label]
        df_investigate.to_csv('/home/yuchen/shapes/notebooks/' + filename + '.csv')

    df_AllSub = df_investigate.copy()

In [ ]:
copy = df_investigate.copy()
df_s = pd.DataFrame({})
for subj in ['12-005','51-009','52-001','All']:
    if subj=='All':
        df_t = df_AllSub.copy()
    else:   
        df_t = copy[copy.subject == subj]
    df_t['amp'] = (df_t['amp'] - df_t['amp'].mean()) / df_t['amp'].std()
    df_t['single_dti'] = (df_t['single_dti'] - df_t['single_dti'].mean()) / df_t['single_dti'].std()
    df_t['single_dtf'] = (df_t['single_dtf'] - df_t['single_dtf'].mean()) / df_t['single_dtf'].std()
    df_t['across'] = (df_t['across'] - df_t['across'].mean()) / df_t['across'].std()
    df_t['along'] = (df_t['along'] - df_t['along'].mean()) / df_t['along'].std()

    
    if subj == 'All':  
        df_AllSub = df_t.copy()
    else:
        df_s = pd.concat([df_s, df_t])
        df_s = df_s.fillna(0)  # for electrode-retinal distance
        
df_investigate = df_s.copy().reset_index(drop=True)

In [ ]:
# df_investigate = df_investigate.rename(columns={"double": "phos_num"})
# df_investigate.to_csv('/home/yuchen/' + 'temp_file' + '.csv')
# df_AllSub = df_AllSub.rename(columns={"double": "phos_num"})
# df_AllSub.to_csv('/home/yuchen/' + 'temp_file_df_AllSub' + '.csv')

In [ ]:
fig, axs= plt.subplots(ncols=5, figsize=(35, 7))
column_lst = ['amp','single_dtf', 'single_dti','across','along']
name_lst = ['Amplitude','Foveal Eccentricity', 'Electrode-Implant Distance','Between-Axonal Distance','Along-Axonal Distance']
for dv in range(5):
    reg = LinearRegression().fit(df_AllSub[column_lst[:dv] + column_lst[dv+1 :]], df_AllSub[['double']])
    y_predicted = reg.predict(df_AllSub[column_lst[:dv] + column_lst[dv+1 :]])
    y = df_AllSub[['double']]-y_predicted

    reg = LinearRegression().fit(df_AllSub[column_lst[:dv] + column_lst[dv+1 :]], df_AllSub[column_lst[dv]])
    y_predicted = reg.predict(df_AllSub[column_lst[:dv] + column_lst[dv+1 :]])
    x = df_AllSub[column_lst[dv]]-y_predicted
    df_AllSub['x'] = x
    df_AllSub['y'] = y
    subject = ['12-005','51-009','52-001']
    marker_lst = ['o','s','^']
    color_lst = ['#1E88E5', '#FFC107','#004D40']
    for i in range(len(subject)):
        temp = df_AllSub[df_AllSub.subject == subject[i]]
        axs[dv].plot(temp['x'],temp['y'],marker_lst[i], color=color_lst[i],alpha=0.6,markersize=13)
    axs[dv].legend(subject, loc='upper right',prop={'size': 20})
    reg = LinearRegression().fit(np.array(x).reshape(-1,1),np.array(y).reshape(-1,1))
    y_pred = reg.predict(np.array(x).reshape(-1,1))
    axs[dv].plot(x, y_pred,'-', color="black",linewidth=2)
#     axs[dv].set(xlabel = name_lst[dv],ylim=(-0.8,1.6),xlim=(-2,3.5))
    axs[0].set(ylabel = 'Number of Phosphenes')
    axs[dv].tick_params(axis='both',  labelsize=18)
    for item in ([axs[dv].xaxis.label, axs[dv].yaxis.label]):
        item.set_fontsize(30)
#     for item in range(1):
#         axs[dv].set_xticklabels(labels = '', fontsize=18)
fig.savefig('/home/yuchen/paper/12f. Double-Electrode Number of Phosphenes.pdf', transparent=True)

In [ ]:
for subj in ['12-005','51-009','52-001','AllSubjects']:
    temp = df_investigate[df_investigate.subject == subj].reset_index(drop=True)
    if subj == '12-005':
        X = temp[['amp','single_dtf','single_dti','across', 'along']]
    elif subj == 'AllSubjects':
        temp = df_AllSub.copy()
        X = temp[['amp','single_dtf','single_dti','across', 'along']]
    else:
        X = temp[['amp','single_dtf','across', 'along']]
    y = temp.double
    X2 = sm.add_constant(X)
    est = sm.OLS(y, X2)
    est2 = est.fit()
    print('subject: ' + subj)
    print(est2.summary())
    print('\ncoefficient')
    [ print(round(x,3), end='\n') for x in est2.params.tolist()[1:]]
    print('\npvalues')
    [ print(round(x,3), end='\n') for x in est2.pvalues.tolist()[1:]]
    print('\nvariance inflation factor: ')
    print([[X.columns[i],variance_inflation_factor(X.values, i)]
                          for i in range(len(X.columns))])
    print('\npartial correlation: ')
    X['double'] = temp['double']
    [print(round(X.pcorr()['double'][i].tolist(), 3), end='\n') for i in X.pcorr()[:-1]]
    print(' ')
    

## 2. pair_electrode shape = single_electrode shape + single_electrode shape

In [ ]:
# calculate shapes 
data = shapes.load_shapes("/home/yuchen/shapes/data/shapes.h5", subjects=['12-005','51-009','52-001'],stim_class=None)


result = pd.DataFrame({})
for i in range(len(data)):
    label_img = skimage.measure.label(data['image'][i]>0)
    regions = regionprops(label_img)
    props = regionprops_table(label_img, properties=('centroid',
                                                     'orientation',
                                                     'major_axis_length',
                                                     'minor_axis_length',
                                                     'area',
                                                    'eccentricity',
                                                    'perimeter'))
    df = pd.DataFrame(props).astype('object')
    df.at[0,'centroid-0'] = df.iloc[:, 0].tolist()  # store centroid-x
    df.at[0,'centroid-1'] = df.iloc[:, 1].tolist()  # store centroid-y
    df.at[0,'orientation'] = df.iloc[:, 2].tolist()  # store orientation
    df.at[0,'major_axis_length'] = df.iloc[:, 3].tolist()  # major
    df.at[0,'minor_axis_length'] = df.iloc[:, 4].tolist()  # minor
    df.at[0,'area'] = df.iloc[:, 5].tolist()  # area
    df.at[0,'eccentricity'] = df.iloc[:, 6].tolist()  
    df.at[0,'perimeter'] = df.iloc[:, 7].tolist()  
    result = pd.concat([result, df.iloc[:1,:]],axis=0)

result = result.rename(columns={"area":"size", "orientation":"orientation_new", "eccentricity":"eccentricity_new" })
data = pd.concat([data,result.reset_index(drop=True)],axis=1)
data_copy = data.copy()

In [ ]:
data = data_copy.copy()

data_double = data[(data['electrode2'] != str())].reset_index(drop=True)
data_single = data[(data['electrode2'] == str())].reset_index(drop=True)
data_single['stim_class'] ='SingleElectrode'
df = data_single[['subject', 'electrode1', 'freq', 'amp1']].drop_duplicates().reset_index(drop=True)
lst = []
lst_size = []
lst_major = []
lst_minor = []
lst_eccentricity = []
lst_orientation = []
lst_perimeter = []

for i in range(len(df)):
    df_temp = data_single[(data_single['subject'] == df.subject[i]) & (data_single['electrode1'] == df.electrode1[i]) & (data_single['freq'] == df.freq[i]) & (data_single['amp1'] == df.amp1[i])].reset_index(drop=True)
    ratio = len(df_temp[df_temp['num_regions']>1]) /len(df_temp)
    
    for row in range(len(df_temp)):
        df_temp['size'][row] = sum(df_temp['size'][row])
        df_temp['major_axis_length'][row] = sum(df_temp['major_axis_length'][row])
        df_temp['minor_axis_length'][row] = sum(df_temp['minor_axis_length'][row])
        df_temp['perimeter'][row] = sum(df_temp['perimeter'][row])
        df_temp['orientation_new'][row] = sum(df_temp['orientation_new'][row])
        
    
    count_length = len(df_temp)
    
    lst_size.append(df_temp['size'].sum()/count_length)
    lst_major.append(df_temp['major_axis_length'].sum()/count_length)
    lst_minor.append(df_temp['minor_axis_length'].sum()/count_length)
    lst_perimeter.append(df_temp['perimeter'].sum()/count_length)
    lst_orientation.append(df_temp['orientation_new'].sum()/count_length)
    
        
df['avg_size'] = lst_size
df['avg_major'] = lst_major
df['avg_minor'] = lst_minor
df['avg_perimeter'] = lst_perimeter
df['avg_orientation'] = lst_orientation


data_single = df.copy()

index_lst = []
for i in range(len(data_double)):
    if not (data_single[(data_single['subject'] == data_double.subject[i]) & (data_single['electrode1'] == data_double.electrode1[i]) & (data_single['freq'] == data_double.freq[i]) & (data_single['amp1'] == data_double.amp1[i])].empty):
        if not (data_single[(data_single['subject'] == data_double.subject[i]) & (data_single['electrode1'] == data_double.electrode2[i]) & (data_single['freq'] == data_double.freq[i]) & (data_single['amp1'] == data_double.amp1[i])].empty):
            index_lst.append(i)

In [ ]:
subject = ['12-005','51-009','52-001']
df_dti = pd.DataFrame({})
for subj in subject:
    lst_e = []
    lst_dtf = []

    s = shapes.subject_params[subj]
    implant,model = shapes.model_from_params(s)
    for i in string.ascii_uppercase[0:6]: 
        for j in range(1,11):
            electrode = i + str(j)
            lst_e.append(electrode)
            lst_dtf.append(math.sqrt(implant[electrode].x**2 +implant[electrode].y**2 ))

    df_o = pd.DataFrame(lst_e, columns=['electrode1'])
    df_o['distance_to_fovea'] = lst_dtf

    if subj == '12-005':
        lst_d = [0,0,0,2,7,7,0,0,5,0,
                    0,0,4,11,13,15,15,3,7,4,
                    0,0,15,16,17,17,19,16,9,4,
                    0,0,16,19,15,17,22,25,13,10,
                    0,0,8,15,14,13,17,23,14,2,
                    0,0,10,15,13,12,9,11,5]
        lst_e = lst_e[:-1]
    elif subj == '51-009':
        lst_e = ['F1','F2','F3','F4','F5','F6',
               'E1','E2','E3','E4','E5','E6',
               'D1','D2','D3','D4','D5','D6','D7',
               'C1','C2','C3','C4','C5','C6','C7','C8',
               'B1','B2','B3','B4','B5','B6','B7','B8','B9','B10', 
               'A1','A2','A3','A4','A5','A6','A7','A8','A9','A10']
        
        lst_d = [0] * len(lst_e)
    else:
        lst_e = ['F1','F2','F4','F5','F6','F7','F8','F9','F10',
                   'E1','E2','E3','E4','E5','E6','E7','E8','E9','E10', 
                   'D1','D2','D3','D4','D5','D6','D7','D8','D9','D10', 
                   'C3','C4','C5','C6','C7','C8','C9','C10', 
                   'B1','B3','B4','B5','B6','B7','B8','B9','B10', 
                   'A1','A2','A3','A4','A5','A6','A7','A8','A9','A10']
        lst_d = [0] * len(lst_e)
    lst_s = [subj] * len(lst_e)

    df_o = df_o[df_o.electrode1.isin(lst_e)]
    df_o['distance_to_implant'] = lst_d
    df_o['subject'] = lst_s
    df_dti = pd.concat([df_dti, df_o])
df_dti = df_dti.reset_index(drop=True)

data_single = data_single.merge(df_dti, how = 'inner', on = ['electrode1','subject'])

### size

In [ ]:
label_list = ['MultiElectrode']
for label in range(1):
    data_double_temp = data_double.iloc[index_lst].reset_index(drop=True)
    data_double_temp = data_double_temp[data_double_temp['stim_class'] == label_list[label]]
    lst = data_double_temp[['electrode1','electrode2','amp1','freq','subject']]
    lst = lst.drop_duplicates().reset_index(drop=True)
    double = []
    single = []
    single1 = []
    single2 = []
    lst_subject = []
    electrode_pair = []
        
    distance = []
    distance_tan = []
    distance_per = []
    same_side = []   # check if both electrodes are at the same side of raphe, 1 for true, 0 for false
    within_range = []  # check if electrode is within 2mm of fovea, 1 for true, 0 for false
    between = []
    along = []
    amp = []
    freq = []
    electrode = []
    inside = []
    single_dti = []
    single_dtf=[]
    for i in range(len(lst)):
        electrode1 = lst.electrode1[i]
        electrode2 = lst.electrode2[i]
        temp1 = data_single[(data_single['electrode1'] == electrode1) &  (data_single['subject'] == lst.subject[i])  & (data_single['amp1'] == lst.amp1[i]) & (data_single['freq'] == lst.freq[i])].reset_index(drop=True)
        temp2 = data_single[(data_single['electrode1'] == electrode2) &  (data_single['subject'] == lst.subject[i])  & (data_single['amp1'] == lst.amp1[i]) & (data_single['freq'] == lst.freq[i])].reset_index(drop=True)
        double_temp = data_double_temp[(data_double_temp['electrode1'] == electrode1) & ((data_double_temp['electrode2'] == electrode2))& (data_double_temp['subject'] == lst.subject[i]) & (data_double_temp['amp1'] == lst.amp1[i]) & (data_double_temp['freq'] == lst.freq[i])].reset_index(drop=True)
        if not temp1.empty and not temp2.empty and not double_temp.empty:
            count = 0
            for j in range(len(double_temp)):
                count += sum(double_temp['size'][j])
            double.append(count/len(double_temp))
            single.append(temp1.avg_size[0] + temp2.avg_size[0])
            single1.append(temp1.avg_size[0])
            single2.append(temp2.avg_size[0])
            
            lst_subject.append(temp1.subject[0])
                
            subject = shapes.subject_params[temp1.subject[0]]
            implant,model = shapes.model_from_params(subject)
                
            e1_x = implant[electrode1].x
            e1_y = implant[electrode1].y
            e2_x = implant[electrode2].x
            e2_y = implant[electrode2].y
            
            single_dti.append(mean([df_dti[(df_dti.electrode1 == lst.electrode1[i]) & 
                                           (df_dti.subject == lst.subject[i])].distance_to_implant.tolist()[0], 
                                    df_dti[(df_dti.electrode1 == lst.electrode2[i]) & 
                                           (df_dti.subject == lst.subject[i])].distance_to_implant.tolist()[0]]))
            
            
            distance.append(math.sqrt((e1_x - e2_x)**2 + (e1_y - e2_y)**2))
            single_dtf.append(mean([math.sqrt(e1_x**2 + e1_y**2), math.sqrt(e2_x**2 + e2_y**2)]))
            
            amp.append(lst.amp1[i])
            freq.append(lst.freq[i])
            electrode_pair.append(electrode1 + str('_') + electrode2)
    
    
    
    df_investigate = pd.DataFrame({    'single':single,
                                   'single1':single1,
                                   'single2':single2,
                                       'double':double, 
                                   'single_dti':single_dti,
                                   'single_dtf':single_dtf,

                                   'distance':distance,
                                   'electrode_pair':electrode_pair,
                                       'subject':lst_subject,
                                   'amp':amp,
                                   'freq':freq
                                      })

    temp = df_investigate[(df_investigate.amp == 2) & (df_investigate.freq == 20) & 
                              (df_investigate.single_dti == 0) &
                              (df_investigate.single_dtf > 2950) & (df_investigate.single_dtf < 3450)]
                          
    temp = temp.groupby(['amp','freq','subject']).mean().reset_index()
    df = pd.DataFrame({})
    for subj in temp.subject:
        df_investigate_temp = df_investigate[df_investigate.subject == subj].reset_index(drop=True)
        df_investigate_temp['single'] = df_investigate_temp['single'] / temp[temp.subject == subj].iloc[0]['single']
        df_investigate_temp['double'] = df_investigate_temp['double'] / temp[temp.subject == subj].iloc[0]['single']
        df = pd.concat([df, df_investigate_temp])
    df_investigate = df.copy()

In [ ]:
df_investigate = pd.DataFrame({    'single':single,
                                   'single1':single1,
                                   'single2':single2,
                                       'double':double, 
                                   'single_dti':single_dti,
                                   'single_dtf':single_dtf,

                                   'distance':distance,
                                   'electrode_pair':electrode_pair,
                                       'subject':lst_subject,
                                   'amp':amp,
                                   'freq':freq
                                      })

temp = df_investigate[(df_investigate.amp == 2) & (df_investigate.freq == 20) ]
                              (df_investigate.single_dti == 0) 
                              (df_investigate.single_dtf > 2950) & (df_investigate.single_dtf < 3450)]



In [ ]:
for subj in ['12-005','51-009','52-001','AllSubjects']:
    temp = df_investigate[df_investigate.subject == subj].reset_index(drop=True)
    if subj == 'AllSubjects':
        temp = df_investigate
    X = temp[['single']]
    y = temp.double
#     X2 = sm.add_constant(X)
#     est = sm.OLS(y, X2)
    est = sm.OLS(y, X)

    est2 = est.fit()
    print('subject: ' + subj)
    print(est2.summary())
    print('\ncoefficient with more digits: ')
    print(est2.params)
    print('\npartial correlation: ')
    print(temp[['single','double']].pcorr())
    print(' ')

### major

In [ ]:
label_list = ['MultiElectrode']
for label in range(1):
    data_double_temp = data_double.iloc[index_lst].reset_index(drop=True)
    data_double_temp = data_double_temp[data_double_temp['stim_class'] == label_list[label]]
    lst = data_double_temp[['electrode1','electrode2','amp1','freq','subject']]
    lst = lst.drop_duplicates().reset_index(drop=True)
    double = []
    single = []
    single1 = []
    single2 = []
    lst_subject = []
    electrode_pair = []
        
    distance = []
    distance_tan = []
    distance_per = []
    same_side = []   # check if both electrodes are at the same side of raphe, 1 for true, 0 for false
    within_range = []  # check if electrode is within 2mm of fovea, 1 for true, 0 for false
    between = []
    along = []
    amp = []
    freq = []
    electrode = []
    inside = []
    single_dti = []
    single_dtf=[]
    for i in range(len(lst)):
        electrode1 = lst.electrode1[i]
        electrode2 = lst.electrode2[i]
        temp1 = data_single[(data_single['electrode1'] == electrode1) &  (data_single['subject'] == lst.subject[i])  & (data_single['amp1'] == lst.amp1[i]) & (data_single['freq'] == lst.freq[i])].reset_index(drop=True)
        temp2 = data_single[(data_single['electrode1'] == electrode2) &  (data_single['subject'] == lst.subject[i])  & (data_single['amp1'] == lst.amp1[i]) & (data_single['freq'] == lst.freq[i])].reset_index(drop=True)
        double_temp = data_double_temp[(data_double_temp['electrode1'] == electrode1) & ((data_double_temp['electrode2'] == electrode2))& (data_double_temp['subject'] == lst.subject[i]) & (data_double_temp['amp1'] == lst.amp1[i]) & (data_double_temp['freq'] == lst.freq[i])].reset_index(drop=True)
        if not temp1.empty and not temp2.empty and not double_temp.empty:
            count = 0
            for j in range(len(double_temp)):
                count += sum(double_temp['major_axis_length'][j])
            double.append(count/len(double_temp))

            single.append(temp1.avg_major[0] + temp2.avg_major[0])
            single1.append(temp1.avg_major[0])
            single2.append(temp2.avg_major[0])
            
            lst_subject.append(temp1.subject[0])
                
            subject = shapes.subject_params[temp1.subject[0]]
            implant,model = shapes.model_from_params(subject)
                
            e1_x = implant[electrode1].x
            e1_y = implant[electrode1].y
            e2_x = implant[electrode2].x
            e2_y = implant[electrode2].y
            
            single_dti.append(mean([df_dti[(df_dti.electrode1 == lst.electrode1[i]) & 
                                           (df_dti.subject == lst.subject[i])].distance_to_implant.tolist()[0], 
                                    df_dti[(df_dti.electrode1 == lst.electrode2[i]) & 
                                           (df_dti.subject == lst.subject[i])].distance_to_implant.tolist()[0]]))
            
            distance.append(math.sqrt((e1_x - e2_x)**2 + (e1_y - e2_y)**2))
            single_dtf.append(mean([math.sqrt(e1_x**2 + e1_y**2), math.sqrt(e2_x**2 + e2_y**2)]))
            
            amp.append(lst.amp1[i])
            freq.append(lst.freq[i])
            electrode_pair.append(electrode1 + '_' + electrode2)
    
    
    
    df_investigate = pd.DataFrame({    'single':single,
                                   'single1':single1,
                                   'single2':single2,
                                       'double':double, 
                                   'single_dti':single_dti,
                                   'single_dtf':single_dtf,

                                   'distance':distance,
                                   'electrode_pair':electrode_pair,
                                       'subject':lst_subject,
                                   'amp':amp,
                                   'freq':freq
                                      })

    temp = df_investigate[(df_investigate.amp == 2) & (df_investigate.freq == 20) & 
                              (df_investigate.single_dti == 0) &
                              (df_investigate.single_dtf > 2950) & (df_investigate.single_dtf < 3450)]
                          
    temp = temp.groupby(['amp','freq','subject']).mean().reset_index()
    df = pd.DataFrame({})
    for subj in temp.subject:
        df_investigate_temp = df_investigate[df_investigate.subject == subj].reset_index(drop=True)
        df_investigate_temp['single'] = df_investigate_temp['single'] / temp[temp.subject == subj].iloc[0]['single']
        df_investigate_temp['double'] = df_investigate_temp['double'] / temp[temp.subject == subj].iloc[0]['single']
        df = pd.concat([df, df_investigate_temp])
    df_investigate = df.copy()

In [ ]:
for subj in ['12-005','51-009','52-001','AllSubjects']:
    temp = df_investigate[df_investigate.subject == subj].reset_index(drop=True)
    if subj == 'AllSubjects':
        temp = df_investigate
    X = temp[['single']]
    y = temp.double
    est = sm.OLS(y, X)
    
#     X2 = sm.add_constant(X)
#     est = sm.OLS(y, X2)
    est2 = est.fit()
    print('subject: ' + subj)
    print(est2.summary())
    print('\ncoefficient with more digits: ')
    print(est2.params)
    print('\npartial correlation: ')
    print(temp[['single','double']].pcorr())
    print(' ')

### minor

In [ ]:
label_list = ['MultiElectrode']
for label in range(1):
    data_double_temp = data_double.iloc[index_lst].reset_index(drop=True)
    data_double_temp = data_double_temp[data_double_temp['stim_class'] == label_list[label]]
    lst = data_double_temp[['electrode1','electrode2','amp1','freq','subject']]
    lst = lst.drop_duplicates().reset_index(drop=True)
    double = []
    single = []
    single1 = []
    single2 = []
    lst_subject = []
    electrode_pair = []
        
    distance = []
    distance_tan = []
    distance_per = []
    same_side = []   # check if both electrodes are at the same side of raphe, 1 for true, 0 for false
    within_range = []  # check if electrode is within 2mm of fovea, 1 for true, 0 for false
    between = []
    along = []
    amp = []
    freq = []
    electrode = []
    inside = []
    single_dti = []
    single_dtf=[]
    for i in range(len(lst)):
        electrode1 = lst.electrode1[i]
        electrode2 = lst.electrode2[i]
        temp1 = data_single[(data_single['electrode1'] == electrode1) &  (data_single['subject'] == lst.subject[i])  & (data_single['amp1'] == lst.amp1[i]) & (data_single['freq'] == lst.freq[i])].reset_index(drop=True)
        temp2 = data_single[(data_single['electrode1'] == electrode2) &  (data_single['subject'] == lst.subject[i])  & (data_single['amp1'] == lst.amp1[i]) & (data_single['freq'] == lst.freq[i])].reset_index(drop=True)
        double_temp = data_double_temp[(data_double_temp['electrode1'] == electrode1) & ((data_double_temp['electrode2'] == electrode2))& (data_double_temp['subject'] == lst.subject[i]) & (data_double_temp['amp1'] == lst.amp1[i]) & (data_double_temp['freq'] == lst.freq[i])].reset_index(drop=True)
        if not temp1.empty and not temp2.empty and not double_temp.empty:
            count = 0
            for j in range(len(double_temp)):
                count += sum(double_temp['minor_axis_length'][j])
            double.append(count/len(double_temp))
            single.append(temp1.avg_minor[0] + temp2.avg_minor[0])
            single1.append(temp1.avg_minor[0])
            single2.append(temp2.avg_minor[0])
            
            lst_subject.append(temp1.subject[0])
                
            subject = shapes.subject_params[temp1.subject[0]]
            implant,model = shapes.model_from_params(subject)
                
            e1_x = implant[electrode1].x
            e1_y = implant[electrode1].y
            e2_x = implant[electrode2].x
            e2_y = implant[electrode2].y
            
            amp.append(lst.amp1[i])
            freq.append(lst.freq[i])
            
            single_dti.append(mean([df_dti[(df_dti.electrode1 == lst.electrode1[i]) & 
                                           (df_dti.subject == lst.subject[i])].distance_to_implant.tolist()[0], 
                                    df_dti[(df_dti.electrode1 == lst.electrode2[i]) & 
                                           (df_dti.subject == lst.subject[i])].distance_to_implant.tolist()[0]]))
                
            
            distance.append(math.sqrt((e1_x - e2_x)**2 + (e1_y - e2_y)**2))
            single_dtf.append(mean([math.sqrt(e1_x**2 + e1_y**2), math.sqrt(e2_x**2 + e2_y**2)]))
            electrode_pair.append(electrode1 + str('_') + electrode2)
    
    df_investigate = pd.DataFrame({    'single':single,
                                   'single1':single1,
                                   'single2':single2,
                                       'double':double, 
                                   'single_dti':single_dti,
                                   'single_dtf':single_dtf,

                                   'distance':distance,
                                   'electrode_pair':electrode_pair,
                                       'subject':lst_subject,
                                   'amp':amp,
                                   'freq':freq
                                      })

    temp = df_investigate[(df_investigate.amp == 2) & (df_investigate.freq == 20) & 
                              (df_investigate.single_dti == 0) &
                              (df_investigate.single_dtf > 2950) & (df_investigate.single_dtf < 3450)]
                          
    temp = temp.groupby(['amp','freq','subject']).mean().reset_index()
    df = pd.DataFrame({})
    for subj in temp.subject:
        df_investigate_temp = df_investigate[df_investigate.subject == subj].reset_index(drop=True)
        df_investigate_temp['single'] = df_investigate_temp['single'] / temp[temp.subject == subj].iloc[0]['single']
        df_investigate_temp['double'] = df_investigate_temp['double'] / temp[temp.subject == subj].iloc[0]['single']
        df = pd.concat([df, df_investigate_temp])
    df_investigate = df.copy()

In [ ]:
for subj in ['12-005','51-009','52-001','AllSubjects']:
    temp = df_investigate[df_investigate.subject == subj].reset_index(drop=True)
    if subj == 'AllSubjects':
        temp = df_investigate
    X = temp[['single']]
    y = temp.double
    est = sm.OLS(y, X)
#     X2 = sm.add_constant(X)
#     est = sm.OLS(y, X2)
    est2 = est.fit()
    print('subject: ' + subj)
    print(est2.summary())
    print('\ncoefficient with more digits: ')
    print(est2.params)
    print('\npartial correlation: ')
    print(temp[['single','double']].pcorr())
    print(' ')

### perimeter

In [ ]:
label_list = ['MultiElectrode']
for label in range(1):
    data_double_temp = data_double.iloc[index_lst].reset_index(drop=True)
    data_double_temp = data_double_temp[data_double_temp['stim_class'] == label_list[label]]
    lst = data_double_temp[['electrode1','electrode2','amp1','freq','subject']]
    lst = lst.drop_duplicates().reset_index(drop=True)
    double = []
    single = []
    single1 = []
    single2 = []
    lst_subject = []
    electrode_pair = []
    distance = []
    distance_tan = []
    distance_per = []
    between = []
    along = []
    same_side = []
    amp = []
    freq = []
    inside = []
    electrode = []
    single_dti = []
    single_dtf=[]
    for i in range(len(lst)):
        electrode1 = lst.electrode1[i]
        electrode2 = lst.electrode2[i]
        temp1 = data_single[(data_single['electrode1'] == electrode1) &  (data_single['subject'] == lst.subject[i])  & (data_single['amp1'] == lst.amp1[i]) & (data_single['freq'] == lst.freq[i])].reset_index(drop=True)
        temp2 = data_single[(data_single['electrode1'] == electrode2) &  (data_single['subject'] == lst.subject[i])  & (data_single['amp1'] == lst.amp1[i]) & (data_single['freq'] == lst.freq[i])].reset_index(drop=True)
        double_temp = data_double_temp[(data_double_temp['electrode1'] == electrode1) & ((data_double_temp['electrode2'] == electrode2))& (data_double_temp['subject'] == lst.subject[i]) & (data_double_temp['amp1'] == lst.amp1[i]) & (data_double_temp['freq'] == lst.freq[i])].reset_index(drop=True)
        if not temp1.empty and not temp2.empty and not double_temp.empty:
            count = 0
            for j in range(len(double_temp)):
                count += sum(double_temp['perimeter'][j])
            double.append(count/len(double_temp))
            single.append(temp1.avg_perimeter[0] + temp2.avg_perimeter[0])
            single1.append(temp1.avg_perimeter[0])
            single2.append(temp2.avg_perimeter[0])
            
            lst_subject.append(temp1.subject[0])
                
            subject = shapes.subject_params[temp1.subject[0]]
            implant,model = shapes.model_from_params(subject)
                
            e1_x = implant[electrode1].x
            e1_y = implant[electrode1].y
            e2_x = implant[electrode2].x
            e2_y = implant[electrode2].y
                
            amp.append(lst.amp1[i])
            freq.append(lst.freq[i])
            
            electrode_pair.append(electrode1+'_'+electrode2)
            
            single_dti.append(mean([df_dti[(df_dti.electrode1 == lst.electrode1[i]) & 
                                           (df_dti.subject == lst.subject[i])].distance_to_implant.tolist()[0], 
                                    df_dti[(df_dti.electrode1 == lst.electrode2[i]) & 
                                           (df_dti.subject == lst.subject[i])].distance_to_implant.tolist()[0]]))
                
            
            distance.append(math.sqrt((e1_x - e2_x)**2 + (e1_y - e2_y)**2))
            single_dtf.append(mean([math.sqrt(e1_x**2 + e1_y**2), math.sqrt(e2_x**2 + e2_y**2)]))
    
    
    df_investigate = pd.DataFrame({    'single':single,
                                   'single1':single1,
                                   'single2':single2,
                                       'double':double, 
                                   'single_dti':single_dti,
                                   'single_dtf':single_dtf,
                                   'electrode_pair':electrode_pair,

                                   'distance':distance,
                                       'subject':lst_subject,
                                   'amp':amp,
                                   'freq':freq
                                      })

    temp = df_investigate[(df_investigate.amp == 2) & (df_investigate.freq == 20) & 
                              (df_investigate.single_dti == 0) &
                              (df_investigate.single_dtf > 2950) & (df_investigate.single_dtf < 3450)]
                       
    temp = temp.groupby(['amp','freq','subject']).mean().reset_index()
    df = pd.DataFrame({})
    for subj in temp.subject:
        df_investigate_temp = df_investigate[df_investigate.subject == subj].reset_index(drop=True)
        df_investigate_temp['single'] = df_investigate_temp['single'] / temp[temp.subject == subj].iloc[0]['single']
        df_investigate_temp['double'] = df_investigate_temp['double'] / temp[temp.subject == subj].iloc[0]['single']
        df = pd.concat([df, df_investigate_temp])
    df_investigate = df.copy()

In [ ]:
for subj in ['12-005','51-009','52-001','AllSubjects']:
    temp = df_investigate[df_investigate.subject == subj].reset_index(drop=True)
    if subj == 'AllSubjects':
        temp = df_investigate
    X = temp[['single']]
    y = temp.double
    est = sm.OLS(y, X)
#     X2 = sm.add_constant(X)
#     est = sm.OLS(y, X2)
    est2 = est.fit()
    print('subject: ' + subj)
    print(est2.summary())
    print('\ncoefficient with more digits: ')
    print(est2.params)
    print('\npartial correlation: ')
    print(temp[['single','double']].pcorr())
    print(' ')

### orientation

In [ ]:
label_list = ['MultiElectrode']
for label in range(1):
    data_double_temp = data_double.iloc[index_lst].reset_index(drop=True)
    data_double_temp = data_double_temp[data_double_temp['stim_class'] == label_list[label]]
    lst = data_double_temp[['electrode1','electrode2','amp1','freq','subject']]
    lst = lst.drop_duplicates().reset_index(drop=True)
    double = []
    single = []
    single1 = []
    single2 = []
    lst_subject = []
    between = []
    along = []
    distance = []
    distance_tan = []
    distance_per = []
    same_side = []
    inside = []
    amp = []
    freq = []
    electrode = []
    for i in range(len(lst)):
        electrode1 = lst.electrode1[i]
        electrode2 = lst.electrode2[i]
        temp1 = data_single[(data_single['electrode1'] == electrode1) &  (data_single['subject'] == lst.subject[i])  & (data_single['amp1'] == lst.amp1[i]) & (data_single['freq'] == lst.freq[i])].reset_index(drop=True)
        temp2 = data_single[(data_single['electrode1'] == electrode2) &  (data_single['subject'] == lst.subject[i])  & (data_single['amp1'] == lst.amp1[i]) & (data_single['freq'] == lst.freq[i])].reset_index(drop=True)
        double_temp = data_double_temp[(data_double_temp['electrode1'] == electrode1) & ((data_double_temp['electrode2'] == electrode2))& (data_double_temp['subject'] == lst.subject[i]) & (data_double_temp['amp1'] == lst.amp1[i]) & (data_double_temp['freq'] == lst.freq[i])].reset_index(drop=True)
        if not temp1.empty and not temp2.empty and not double_temp.empty:
            count = 0
            for j in range(len(double_temp)):
                count += mean(double_temp['orientation_new'][j])
            double.append(count/len(double_temp))
            single.append(mean([temp1.avg_orientation[0], temp2.avg_orientation[0]]))
            single1.append(temp1.avg_orientation[0])
            single2.append(temp2.avg_orientation[0])
            
            lst_subject.append(temp1.subject[0])
                
            subject = shapes.subject_params[temp1.subject[0]]
            implant,model = shapes.model_from_params(subject)
                
            e1_x = implant[electrode1].x
            e1_y = implant[electrode1].y
            e2_x = implant[electrode2].x
            e2_y = implant[electrode2].y

            distance.append(math.sqrt((e1_x - e2_x)**2 + (e1_y - e2_y)**2))
            
            amp.append(lst.amp1[i])
            freq.append(lst.freq[i])

    df_investigate = pd.DataFrame({    'single':single,
                                   'single1':single1,
                                   'single2':single2,
                                       'double':double, 
                                   'distance':distance,
                                       'subject':lst_subject,
                                   'amp':amp,
                                   'freq':freq
                                      })
    for subj in ['12-005', '51-009', '52-001']:
        df_investigate_temp = df_investigate[df_investigate.subject == subj].reset_index(drop=True)
        filename = subj + label_list[label] 
        df_investigate_temp.to_csv('/home/yuchen/shapes/notebooks/' + filename + '.csv')
    filename = label_list[label]
    df_investigate.to_csv('/home/yuchen/shapes/notebooks/' + filename + '.csv')

In [ ]:
for subj in ['12-005','51-009','52-001','AllSubjects']:
    temp = df_investigate[df_investigate.subject == subj].reset_index(drop=True)
    if subj == 'AllSubjects':
        temp = df_investigate
    X = temp[['single']]
    y = temp.double
    est = sm.OLS(y, X)
    est2 = est.fit()
    print('subject: ' + subj)
    print(est2.summary())
    print('\ncoefficient with more digits: ')
    print(est2.params)
    print('\npartial correlation: ')
    print(temp[['single','double']].pcorr())
    print(' ')

### phosphene number 

In [ ]:
data = data_copy.copy()

data_double = data[(data['electrode2'] != str())].reset_index(drop=True)
data_single = data[(data['electrode2'] == str())].reset_index(drop=True)


subject = ['12-005','51-009','52-001']
df_dti = pd.DataFrame({})
for subj in subject:
    lst_e = []
    lst_dtf = []

    s = shapes.subject_params[subj]
    implant,model = shapes.model_from_params(s)
    for i in string.ascii_uppercase[0:6]: 
        for j in range(1,11):
            electrode = i + str(j)
            lst_e.append(electrode)
            lst_dtf.append(math.sqrt(implant[electrode].x**2 +implant[electrode].y**2 ))

    df_o = pd.DataFrame(lst_e, columns=['electrode1'])
    df_o['distance_to_fovea'] = lst_dtf

    if subj == '12-005':
        lst_d = [0,0,0,2,7,7,0,0,5,0,
                    0,0,4,11,13,15,15,3,7,4,
                    0,0,15,16,17,17,19,16,9,4,
                    0,0,16,19,15,17,22,25,13,10,
                    0,0,8,15,14,13,17,23,14,2,
                    0,0,10,15,13,12,9,11,5]
        lst_e = lst_e[:-1]
    elif subj == '51-009':
        lst_e = ['F1','F2','F3','F4','F5','F6',
               'E1','E2','E3','E4','E5','E6',
               'D1','D2','D3','D4','D5','D6','D7',
               'C1','C2','C3','C4','C5','C6','C7','C8',
               'B1','B2','B3','B4','B5','B6','B7','B8','B9','B10', 
               'A1','A2','A3','A4','A5','A6','A7','A8','A9','A10']
        
        lst_d = [0] * len(lst_e)
    else:
        lst_e = ['F1','F2','F4','F5','F6','F7','F8','F9','F10',
                   'E1','E2','E3','E4','E5','E6','E7','E8','E9','E10', 
                   'D1','D2','D3','D4','D5','D6','D7','D8','D9','D10', 
                   'C3','C4','C5','C6','C7','C8','C9','C10', 
                   'B1','B3','B4','B5','B6','B7','B8','B9','B10', 
                   'A1','A2','A3','A4','A5','A6','A7','A8','A9','A10']
        lst_d = [0] * len(lst_e)
    lst_s = [subj] * len(lst_e)

    df_o = df_o[df_o.electrode1.isin(lst_e)]
    df_o['distance_to_implant'] = lst_d
    df_o['subject'] = lst_s
    df_dti = pd.concat([df_dti, df_o])
df_dti = df_dti.reset_index(drop=True)

data_single = data_single.merge(df_dti, how = 'inner', on = ['electrode1','subject'])

In [ ]:
data_single['stim_class'] ='SingleElectrode'
df = data_single[['subject', 'electrode1', 'freq', 'amp1']].drop_duplicates().reset_index(drop=True)

lst = []
lst_phosphene = []

for i in range(len(df)):
    df_temp = data_single[(data_single['subject'] == df.subject[i]) & (data_single['electrode1'] == df.electrode1[i]) & (data_single['freq'] == df.freq[i]) & (data_single['amp1'] == df.amp1[i])].reset_index(drop=True)
    ratio = len(df_temp[df_temp['num_regions']>1]) /len(df_temp)
    lst.append(ratio)
    lst_phosphene.append(mean(df_temp['num_regions']))
        
df['phosphene'] = lst_phosphene

data_single = df


index_lst = []
for i in range(len(data_double)):
    if not (data_single[(data_single['subject'] == data_double.subject[i]) & (data_single['electrode1'] == data_double.electrode1[i]) & (data_single['freq'] == data_double.freq[i]) & (data_single['amp1'] == data_double.amp1[i])].empty):
        if not (data_single[(data_single['subject'] == data_double.subject[i]) & (data_single['electrode1'] == data_double.electrode2[i]) & (data_single['freq'] == data_double.freq[i]) & (data_single['amp1'] == data_double.amp1[i])].empty):
            index_lst.append(i)
            
label_list = ['MultiElectrode']

for label in range(1):
    data_double_temp = data_double.iloc[index_lst].reset_index(drop=True)
    data_double_temp = data_double_temp[data_double_temp['stim_class'] == label_list[label]]
    lst = data_double_temp[['electrode1','electrode2','amp1','freq','subject']]
    lst = lst.drop_duplicates().reset_index(drop=True)

    double = []
    single = []
    single1 = []
    single2 = []
    electrode_pair = []

    lst_subject = []
    distance = []
    electrode = []

    amp = []
    freq = []
    inside = []
    for i in range(len(lst)):
        electrode1 = lst.electrode1[i]
        electrode2 = lst.electrode2[i]
        temp1 = data_single[(data_single['electrode1'] == electrode1) &  (data_single['subject'] == lst.subject[i])  & (data_single['amp1'] == lst.amp1[i]) & (data_single['freq'] == lst.freq[i])].reset_index(drop=True)
        temp2 = data_single[(data_single['electrode1'] == electrode2) &  (data_single['subject'] == lst.subject[i])  & (data_single['amp1'] == lst.amp1[i]) & (data_single['freq'] == lst.freq[i])].reset_index(drop=True)
        double_temp = data_double_temp[(data_double_temp['electrode1'] == electrode1) & ((data_double_temp['electrode2'] == electrode2))& (data_double_temp['subject'] == lst.subject[i]) & (data_double_temp['amp1'] == lst.amp1[i]) & (data_double_temp['freq'] == lst.freq[i])].reset_index(drop=True)
        if not temp1.empty and not temp2.empty and not double_temp.empty:
            count = 0
            for j in range(len(double_temp)):
                count += double_temp['num_regions'][j]
            double.append(count/len(double_temp))
            single.append(sum([temp1.phosphene[0], temp2.phosphene[0]]))
            single1.append(temp1.phosphene[0])
            single2.append(temp2.phosphene[0])
            
            lst_subject.append(temp1.subject[0])
                
            subject = shapes.subject_params[temp1.subject[0]]
            implant,model = shapes.model_from_params(subject)
            
            amp.append(lst.amp1[i])
            freq.append(lst.freq[i])
            electrode_pair.append(electrode1+'_'+electrode2)

    df_investigate = pd.DataFrame({    'single':single,
                                   'single1':single1,
                                   'single2':single2,
                                       'double':double, 

                                       'subject':lst_subject,
                                   'electrode_pair':electrode_pair,
                                   'amp':amp,
                                   'freq':freq
                                      })
    for subj in ['12-005', '51-009', '52-001']:
        df_investigate_temp = df_investigate[df_investigate.subject == subj].reset_index(drop=True)
        filename = subj + label_list[label] 
        df_investigate_temp.to_csv('/home/yuchen/shapes/notebooks/' + filename + '.csv')
    filename = label_list[label]
    df_investigate.to_csv('/home/yuchen/shapes/notebooks/' + filename + '.csv')

In [ ]:
for subj in ['12-005','51-009','52-001','AllSubjects']:
    temp = df_investigate[df_investigate.subject == subj].reset_index(drop=True)
    if subj == 'AllSubjects':
        temp = df_investigate
    X = temp[['single']]
    y = temp.double
    est = sm.OLS(y, X)
#     X2 = sm.add_constant(X)
#     est = sm.OLS(y, X2)
    est2 = est.fit()
    print('subject: ' + subj)
    print(est2.summary())
    print('\ncoefficient with more digits: ')
    print(est2.params)
    print('\npartial correlation: ')
    print(temp[['single','double']].corr())
    print(' ')